# TF Slim

> TF Slim is a lightweight library built on top of TensorFlow core for defining and training models. TF Slim can be used in conjunction with other TensorFlow low level and high-level libraries such as TF Learn. The TF Slim comes as part of the TensorFlow installation in the package: `tf.contrib.slim`.

**Note:** We shall use TF Slim for learning how to use pre-trained models such as VGG16 and Inception V3 in later chapters.

<br/>

---
#### ⚠️ 📚 Documentation location is on GitHub here [tree/v1.15.0/tensorflow/contrib/slim](https://github.com/tensorflow/tensorflow/tree/v1.15.0/tensorflow/contrib/slim)
---

<br/>

### *From the docs:* What are the various components of TF-Slim?

TF-Slim is composed of several parts which were design to exist independently. These include the following main pieces (explained in detail below).

* [arg_scope](https://www.tensorflow.org/code/tensorflow/contrib/framework/python/ops/arg_scope.py): provides a new scope named `arg_scope` that allows a user to define default arguments for specific operations within that scope.
* [data](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/data/): contains TF-slim's [dataset](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/data/dataset.py) definition, [data providers](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/data/data_provider.py), [parallel_reader](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/data/parallel_reader.py), and [decoding](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/data/data_decoder.py) utilities.
* [evaluation](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/evaluation.py): contains routines for evaluating models.
* [layers](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/layers.py): contains high level layers for building models using tensorflow.
* [learning](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/learning.py): contains routines for training models.
* [losses](https://www.tensorflow.org/code/tensorflow/contrib/losses/python/losses/loss_ops.py): contains commonly used loss functions.
* [metrics](https://www.tensorflow.org/code/tensorflow/contrib/metrics/python/ops/metric_ops.py): contains popular evaluation metrics.
* [nets](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/nets/): contains popular network definitions such as [VGG](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/nets/vgg.py) and [AlexNet](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/nets/alexnet.py)
models.
* [queues](https://www.tensorflow.org/code/tensorflow/contrib/slim/python/slim/queues.py): provides a context manager for easily and safely starting and closing
`QueueRunner`s.
* [regularizers](https://www.tensorflow.org/code/tensorflow/contrib/layers/python/layers/regularizers.py): contains weight regularizers.
* [variables](https://www.tensorflow.org/code/tensorflow/contrib/framework/python/ops/variables.py): provides convenience wrappers for variable creation and manipulation.

### TF Slim Workflow:
1. Create the model using slim layers.
2. Provide the input to the layers to instantiate the model.
3. Use the logits and labels to define the loss.
4. Get the total loss using convenience function `get_total_loss()`.
5. Create an optimizer.
6. Create a training function using convenience function `slim.learning.create_train_op()`, `total_loss` and `optimizer`.
7. Run the training using the convenience function `slim.learning.train()` and training function defined in the previous step.

In [2]:
# Check if `TF Slim` is working:
import tensorflow.contrib.slim as slim
eval_ = slim.evaluation.evaluate_once
print("Yes, TF Slim is working if no exceptions here.")

Yes TF Slim is working if no exceptions here.


In [3]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import slim  # <-- This.
import tensorflow as tf
import os

In [4]:
tf.reset_default_graph()

In [5]:
n_classes = 10  
n_steps = 1000

In [6]:
# Let us get the data.
# (Already downloaded in previous example)
mnist = input_data.read_data_sets(
    os.path.join('.', 'mnist'), 
    one_hot=True
)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
# Note the **conversion to tensor** here!
X_train = mnist.train.images
X_train = tf.convert_to_tensor(X_train)
Y_train = mnist.train.labels
Y_train = tf.convert_to_tensor(Y_train)

### Note on `layers`:
⚠️ 📚 Slim imports layers from `tensorflow.contrib.layers.python.layers`, docs for which can be found [here](https://github.com/tensorflow/tensorflow/tree/v1.15.0/tensorflow/contrib/layers).

In [8]:
# Define a simple MLP with slim-provided layers, including dropout!
def mlp(x):
    net = slim.fully_connected(x, 32, scope='fc1')
    net = slim.dropout(net, 0.5, scope='dropout1')
    net = slim.fully_connected(net, 32, scope='fc2')
    net = slim.dropout(net, 0.5, scope='dropout2')
    net = slim.fully_connected(net, n_classes, activation_fn=None, scope='fc3')
    return net

In [9]:
# Define the model
logits = mlp(X_train)

Instructions for updating:
Please use `layer.__call__` method instead.


In [10]:
# Define the loss functions and get the total loss
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=Y_train)
total_loss = tf.losses.get_total_loss()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# Optimiser:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [12]:
# Train op - created via Slim.
train_op = slim.learning.create_train_op(total_loss, optimizer)

In [13]:
# Run the training.
final_loss = slim.learning.train(
    train_op,
    logdir='./slim_logs',
    number_of_steps=n_steps,
    log_every_n_steps=100
)

print('final loss={}'.format(final_loss))

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path ./slim_logs/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 99: loss = 2.2131 (0.005 sec/step)
INFO:tensorflow:global step 199: loss = 2.1237 (0.005 sec/step)
INFO:tensorflow:global step 299: loss = 2.0299 (0.004 sec/step)
INFO:tensorflow:global step 399: loss = 1.9482 (0.005 sec/step)
INFO:tensorflow:global step 499: loss = 1.8621 (0.004 sec/step)
INFO:tensorflow:global step 599: loss = 1.7977 (0.005 sec/step)
INFO:tensorflow:global step 699: loss = 1.7177 (0.004 sec/step)
INFO:tensorflow:global step 799: loss = 1.6590 (0.005 sec/step)
INFO:tensorflow:global step 899: loss = 1.6019 (0.005 sec/step)
INFO:tensorflow:global step 999: loss = 1.5492 (0.004 sec/step)
INFO:tensorflow:Stopping Training

/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/summary/writer/writer.py:386: UserWarning: Attempting to use a closed FileWriter. The operation will be a noop unless the FileWriter is explicitly reopened.
  warnings.warn("Attempting to use a closed FileWriter. "
